# ComfyUI QuickStart 📓

> Minimal, config-driven installer for ComfyUI models and custom nodes.

This notebook is designed to be:
- Config-driven (add HF/CivitAI/custom nodes in one CONFIG object)
- Robust (retries, clear logs, token-aware)
- Portable (no hardcoded model lists)

Usage: Edit CONFIG, run Apply, verify.


In [ ]:
# Tokens and Config
HF_TOKEN = ""
CIVITAI_TOKEN = ""

SHOW_PROGRESS = True
SKIP_EXISTING = True
INSTALL_NODE_REQUIREMENTS = True

CONFIG = {
    "huggingface_models": [
        {"repo": "black-forest-labs/FLUX.1-dev", "files": ["ae.safetensors"], "dest": "vae"},
        {"repo": "black-forest-labs/FLUX.1-dev", "files": ["flux1-dev.safetensors"], "dest": "unet"},
        {"repo": "comfyanonymous/flux_text_encoders", "files": ["t5xxl_fp16.safetensors", "clip_l.safetensors"], "dest": "clip"},
        {"repo": "YarvixPA/FLUX.1-Fill-dev-GGUF", "files": ["flux1-fill-dev-Q6_K.gguf"], "revision": "ed1b97dfd7faedbeaac08635330dc3ed23e86ea1", "dest": "unet"},
        {"repo": "black-forest-labs/FLUX.1-Fill-dev", "files": ["flux1-fill-dev.safetensors"], "dest": "unet"},
        {"repo": "Comfy-Org/sigclip_vision_384", "files": ["sigclip_vision_patch14_384.safetensors"], "dest": "clip_vision"},
        {"repo": "black-forest-labs/FLUX.1-Redux-dev", "files": ["flux1-redux-dev.safetensors"], "dest": "style_models"},
        {"repo": "lovis93/testllm", "files": ["ae.safetensors"], "revision": "ed9cf1af7465cebca4649157f118e331cf2a084f", "dest": "vae"},
        {"repo": "ali-vilab/ACE_Plus", "files": ["portrait/comfyui_portrait_lora64.safetensors"], "dest": "loras"},
        {"repo": "facebook/sam2-hiera-large", "files": ["sam2_hiera_large.pt"], "dest": "sam2"},
        {"repo": "zer0int/CLIP-GmP-ViT-L-14", "files": ["ViT-L-14-TEXT-detail-improved-hiT-GmP-TE-only-HF.safetensors"], "dest": "clip"},
        {"repo": "alimama-creative/FLUX.1-Turbo-Alpha", "files": ["diffusion_pytorch_model.safetensors"], "dest": "unet"},
    ],
    "civitai_items": [
        {"id": "2207266", "type": "checkpoint", "filename": "fluxtrait_2207266.safetensors"},
        {"id": "2132447", "type": "lora"},
        {"id": "2221628", "type": "lora"},
    ],
    "custom_nodes": [
        {"repo": "https://github.com/ltdrdata/ComfyUI-Manager.git"},
        {"repo": "https://github.com/ltdrdata/ComfyUI-Impact-Pack.git"},
        {"repo": "https://github.com/rgthree/rgthree-comfy.git"},
        {"repo": "https://github.com/kijai/ComfyUI-KJNodes.git"},
        {"repo": "https://github.com/kijai/ComfyUI-Florence2.git"},
        {"repo": "https://github.com/kijai/ComfyUI-segment-anything-2.git"},
        {"repo": "https://github.com/welltop-cn/ComfyUI-TeaCache.git"},
        {"repo": "https://github.com/cubiq/ComfyUI_essentials"},
        {"repo": "https://github.com/lquesada/ComfyUI-Inpaint-CropAndStitch"},
        {"repo": "https://github.com/Starnodes2024/ComfyUI_StarNodes.git"},
        {"repo": "https://github.com/giriss/comfy-image-saver.git"}
    ],
    "dirs": {
        "checkpoints": "/workspace/ComfyUI/models/checkpoints",
        "unet": "/workspace/ComfyUI/models/unet",
        "clip": "/workspace/ComfyUI/models/clip",
        "clip_vision": "/workspace/ComfyUI/models/clip_vision",
        "style_models": "/workspace/ComfyUI/models/style_models",
        "vae": "/workspace/ComfyUI/models/vae",
        "loras": "/workspace/ComfyUI/models/loras",
        "embeddings": "/workspace/ComfyUI/models/embeddings",
        "sam2": "/workspace/ComfyUI/models/sam2",
        "custom_nodes": "/workspace/ComfyUI/custom_nodes",
    }
}

print("✅ CONFIG ready.")


In [ ]:
# Helpers
import os, subprocess, shutil, json, time, re, glob, requests

def run(cmd: str, timeout: int = 1800) -> tuple[bool, str]:
    is_curl = cmd.strip().startswith("curl ") or " curl " in cmd
    wants_stream = SHOW_PROGRESS and is_curl
    if wants_stream:
        print(f"$ {cmd}")
        try:
            p = subprocess.run(cmd, shell=True, text=True, timeout=timeout)
            return (p.returncode == 0), ""
        except subprocess.TimeoutExpired:
            return False, f"timeout after {timeout}s: {cmd}"
    try:
        p = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
        out = (p.stdout or '').strip()
        err = (p.stderr or '').strip()
        if p.returncode == 0:
            if out:
                print(out)
            return True, out
        if err:
            print(err)
        return False, err
    except subprocess.TimeoutExpired:
        return False, f"timeout after {timeout}s: {cmd}"


def ensure_dirs():
    paths = CONFIG.get("dirs", {})
    if paths:
        run("mkdir -p " + " ".join(paths.values()))


def ensure_hf_cli():
    run("pip install -q --upgrade huggingface_hub[cli] hf_transfer")
    os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'
    hf = shutil.which('hf') or shutil.which('huggingface-cli')
    if not shutil.which('hf') and hf:
        run(f"ln -sf {hf} /usr/local/bin/hf || true")
    if HF_TOKEN:
        os.environ['HUGGINGFACE_HUB_TOKEN'] = HF_TOKEN
        os.environ['HF_TOKEN'] = HF_TOKEN
        run("hf whoami || true")


def file_exists_nonempty(path: str) -> bool:
    try:
        return os.path.isfile(path) and os.path.getsize(path) > 0
    except Exception:
        return False


def hf_download(repo: str, files: list[str], dest: str, revision: str | None = None) -> None:
    run(f"mkdir -p '{dest}'")
    for fpath in files:
        rev = revision or "main"
        url = f"https://huggingface.co/{repo}/resolve/{rev}/{fpath}"
        out_path = os.path.join(dest, os.path.basename(fpath))
        if SKIP_EXISTING and file_exists_nonempty(out_path):
            print(f"↪️ Skipping existing: {os.path.basename(out_path)}")
            continue
        # Preflight check to give clear errors for gated/missing files
        hdr = f"-H 'Authorization: Bearer {HF_TOKEN}'" if HF_TOKEN else ""
        code_ok, code_out = run(f"bash -lc \"curl -s -o /dev/null -w '%{{http_code}}' -L -I {hdr} '{url}'\"")
        http_code = (code_out or '').strip()
        if code_ok and http_code in {"401","403"}:
            print(f"❌ Access denied ({http_code}) for {repo}/{fpath}@{rev}. Token may lack access or you must accept the model license: https://huggingface.co/{repo}")
            continue
        if code_ok and http_code == "404":
            print(f"❌ Not found (404): {repo}/{fpath}@{rev}. Check filename or revision.")
            continue
        print(f"📥 Downloading {os.path.basename(fpath)} from {repo}...")
        if SHOW_PROGRESS:
            ok, _ = run(f"curl -L --fail --retry 5 --retry-delay 2 --retry-all-errors -C - --progress-bar {hdr} --output '{out_path}' '{url}'")
            if ok:
                print(f"✅ Downloaded: {os.path.basename(fpath)}")
                continue
        ensure_hf_cli()
        rev_flag = f" --revision {revision}" if revision else ""
        ok, _ = run(f"hf download {repo} {fpath}{rev_flag} --local-dir '{dest}'")
        if not ok:
            print(f"❌ HF download failed: {repo}::{fpath}")
        else:
            # Flatten nested subfolders if hf created any
            if "/" in fpath:
                src_path = os.path.join(dest, fpath)
                flat_path = os.path.join(dest, os.path.basename(fpath))
                if os.path.exists(src_path):
                    try:
                        shutil.move(src_path, flat_path)
                        parent_dir = os.path.dirname(src_path)
                        try:
                            os.rmdir(parent_dir)
                        except Exception:
                            pass
                    except Exception as move_err:
                        print(f"⚠️ Flatten warning: {src_path} -> {flat_path}: {move_err}")
            print(f"✅ Downloaded: {os.path.basename(fpath)}")


def civitai_resolve_version(mid_or_vid: str, token: str | None) -> tuple[str | None, dict]:
    headers = {"Authorization": f"Bearer {token}"} if token else {}
    # Fast path: if it's clearly a numeric version id, just return it and skip API
    if isinstance(mid_or_vid, str) and mid_or_vid.isdigit():
        return mid_or_vid, {}
    try:
        r = requests.get(
            f"https://civitai.com/api/v1/model-versions/{mid_or_vid}",
            headers=headers,
            timeout=8,
        )
        if r.status_code == 200:
            data = r.json()
            return str(data.get("id")), data
    except Exception:
        pass
    try:
        r = requests.get(
            f"https://civitai.com/api/v1/models/{mid_or_vid}",
            headers=headers,
            timeout=8,
        )
        if r.status_code == 200:
            m = r.json()
            versions = m.get("modelVersions") or []
            if versions:
                vid = str(versions[0].get("id"))
                try:
                    rv = requests.get(
                        f"https://civitai.com/api/v1/model-versions/{vid}",
                        headers=headers,
                        timeout=8,
                    )
                    return vid, (rv.json() if rv.status_code == 200 else {})
                except Exception:
                    return vid, {}
    except Exception:
        pass
    return None, {}


def civitai_pick_file(version_data: dict) -> tuple[str | None, str]:
    files = version_data.get("files") or []
    primary = next((f for f in files if f.get("primary")), files[0] if files else None)
    if not primary:
        return None, ""
    return primary.get("downloadUrl"), primary.get("name") or ""


def civitai_dest_for_type(model_type: str) -> str:
    t = (model_type or '').lower()
    d = CONFIG.get('dirs', {})
    if t in {"lora","locon","lycoris"}: return d.get('loras')
    if t in {"textualinversion","embedding","textual_inversion"}: return d.get('embeddings')
    return d.get('checkpoints')


def sanitize_filename(name: str) -> str:
    safe_chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-_."
    return ''.join(c for c in name if c in safe_chars) or "civitai_file"


def civitai_download(item: dict):
    vid_or_mid = str(item.get('id'))
    token = CIVITAI_TOKEN or None
    vid, data = civitai_resolve_version(vid_or_mid, token)
    if not vid:
        print(f"❌ Could not resolve CivitAI id {vid_or_mid}")
        return

    # Try to pick URL from API payload; if missing, fall back to direct download URL
    url, name = civitai_pick_file(data) if data else (None, "")
    if not url:
        url = f"https://civitai.com/api/download/models/{vid}"
        name = item.get('filename') or f"civitai_{vid}.safetensors"

    model = data.get('model') if isinstance(data, dict) else None
    mtype = item.get('type') or ((model or {}).get('type') if model else None)
    dest = civitai_dest_for_type(mtype) or CONFIG['dirs']['checkpoints']

    override_name = item.get('filename')
    if override_name:
        filename = sanitize_filename(override_name)
    else:
        mname = ((model or {}).get('name') or 'civitai')
        prefix = re.sub(r'[^A-Za-z0-9_-]+', '', mname)[:16] or 'civitai'
        ext = (name.rsplit('.', 1)[-1] if '.' in name else 'safetensors')
        filename = sanitize_filename(f"{prefix}_{vid}.{ext}")

    run(f"mkdir -p '{dest}'")

    dl = url
    if 'civitai.com/api/download/models' in dl and token:
        sep = '&' if '?' in dl else '?'
        dl = f"{dl}{sep}token={token}"

    out_path = os.path.join(dest, filename)
    if SKIP_EXISTING and file_exists_nonempty(out_path):
        print(f"↪️ Skipping existing: {os.path.basename(out_path)}")
        return

    print(f"📥 Downloading CivitAI model {vid_or_mid} as {filename}...")
    ok, _ = run(f"curl -L --fail --retry 5 --retry-delay 2 --retry-all-errors -C - --progress-bar --output '{out_path}' '{dl}'")
    if not ok:
        print(f"❌ Download failed for CivitAI {vid_or_mid}")
    else:
        print(f"✅ Downloaded: {filename}")


def normalize_model_dirs():
    names = ["checkpoints","unet","clip","clip_vision","style_models","vae","loras","embeddings","sam2"]
    for name in names:
        base = CONFIG.get('dirs', {}).get(name)
        if not base or not os.path.isdir(base):
            continue
        try:
            for entry in os.listdir(base):
                entry_path = os.path.join(base, entry)
                if os.path.isdir(entry_path):
                    for f in os.listdir(entry_path):
                        src = os.path.join(entry_path, f)
                        dst = os.path.join(base, f)
                        if os.path.isfile(src) and not os.path.exists(dst):
                            try:
                                shutil.move(src, dst)
                            except Exception as me:
                                print(f"⚠️ Normalize warning: {src} -> {dst}: {me}")
                    try:
                        os.rmdir(entry_path)
                    except Exception:
                        pass
        except Exception as e:
            print(f"⚠️ Normalize warning for {name}: {e}")

print("✅ Helpers ready.")


In [ ]:
# Apply Configuration
print("🚀 Starting ComfyUI setup...")
ensure_dirs()

# Resolve dest keys to absolute paths
def resolve_dest(dest_key_or_path: str | None) -> str:
    if not dest_key_or_path:
        return CONFIG['dirs']['checkpoints']
    if isinstance(dest_key_or_path, str) and dest_key_or_path.startswith("/"):
        return dest_key_or_path
    return CONFIG['dirs'].get(dest_key_or_path, CONFIG['dirs']['checkpoints'])

# git check
have_git = shutil.which('git') is not None
if not have_git and CONFIG.get('custom_nodes'):
    print("⚠️ git not found; custom nodes cloning will be skipped.")

# HF models
print("\n📥 HuggingFace models...")
for item in CONFIG.get("huggingface_models", []):
    repo = item.get("repo")
    files = item.get("files") or []
    dest = resolve_dest(item.get("dest"))
    revision = item.get("revision")
    if not repo or not files:
        print(f"⚠️ Skipping invalid HF entry: {item}")
        continue
    print(f"➡️ HF: {repo}{'@'+revision if revision else ''} -> {dest}")
    hf_download(repo, files, dest, revision=revision)

# CivitAI
items = CONFIG.get("civitai_items", [])
small_first = [it for it in items if str(it.get('type','')).lower() != 'checkpoint']
checkpoints = [it for it in items if str(it.get('type','')).lower() == 'checkpoint']

if small_first:
    print(f"\n📥 CivitAI LoRAs/Embeddings ({len(small_first)})...")
    for it in small_first:
        try:
            if 'id' not in it:
                print(f"⚠️ Skipping invalid CivitAI entry: {it}")
                continue
            print(f"   • {it}")
            civitai_download(it)
        except Exception as e:
            print(f"❌ CivitAI error: {e}")

if checkpoints:
    print(f"\n📥 CivitAI Checkpoints ({len(checkpoints)})...")
    for it in checkpoints:
        try:
            if 'id' not in it:
                print(f"⚠️ Skipping invalid CivitAI entry: {it}")
                continue
            print(f"   • {it}")
            civitai_download(it)
        except Exception as e:
            print(f"❌ CivitAI error: {e}")

# Custom nodes
print(f"\n🔧 Custom Nodes ({len(CONFIG.get('custom_nodes', []))})...")
if CONFIG.get("custom_nodes") and have_git:
    base = resolve_dest('custom_nodes')
    run(f"mkdir -p '{base}'")
    for node in CONFIG['custom_nodes']:
        repo = node.get('repo')
        if not repo:
            print(f"⚠️ Skipping custom node without repo: {node}")
            continue
        name = repo.rstrip('/').split('/')[-1].replace('.git','')
        path = os.path.join(base, name)
        if os.path.isdir(path):
            print(f"🔄 Updating {name}")
            run(f"cd '{path}' && git pull --ff-only | cat")
        else:
            print(f"⬇️ Cloning {name}")
            run(f"git clone '{repo}' '{path}' | cat")
        if INSTALL_NODE_REQUIREMENTS:
            req_file = os.path.join(path, 'requirements.txt')
            if os.path.exists(req_file):
                print(f"📦 Installing requirements for {name}")
                run(f"pip install -r '{req_file}'")
            else:
                print(f"ℹ️ No requirements.txt found for {name}")
else:
    if not have_git:
        print("⚠️ git not found - custom nodes will be skipped")
    if not CONFIG.get('custom_nodes'):
        print("ℹ️ No custom nodes configured")

normalize_model_dirs()
print("\n✅ ComfyUI setup complete!")


In [ ]:
# Verify Downloads
import os, glob

print("🔍 VERIFICATION REPORT")
print("="*50)


def list_dir(label, path, patterns=("*.safetensors","*.pt","*.ckpt","*.bin","*.gguf")):
    if not path or not os.path.exists(path):
        print(f"❌ {label}: Directory not found")
        return
    files = []
    for pat in patterns:
        files.extend(glob.glob(os.path.join(path, pat)))
    if not files:
        print(f"❌ {label}: No files found")
        return

    valid = []
    broken = []
    for f in files:
        try:
            if os.path.isfile(f):
                sz = os.path.getsize(f)
                valid.append((f, sz))
            elif os.path.islink(f) and not os.path.exists(f):
                broken.append(f)
        except FileNotFoundError:
            broken.append(f)
        except Exception:
            broken.append(f)

    total_mb = sum(sz for _, sz in valid) / (1024*1024)
    print(f"✅ {label}: {len(valid)} files, {total_mb:.1f} MB")
    for f, sz in sorted(valid)[:5]:
        print(f"   - {os.path.basename(f)} ({sz/(1024*1024):.1f} MB)")
    if len(valid) > 5:
        print(f"   ... +{len(valid)-5} more files")
    if broken:
        print(f"⚠️ {label}: {len(broken)} broken entries (dangling symlinks or missing). Skipping.")

base = CONFIG.get('dirs', {})
list_dir("Checkpoints", base.get('checkpoints'))
list_dir("UNet", base.get('unet'))
list_dir("CLIP", base.get('clip'))
list_dir("CLIP Vision", base.get('clip_vision'))
list_dir("Style Models", base.get('style_models'))
list_dir("VAE", base.get('vae'))
list_dir("LoRAs", base.get('loras'))
list_dir("Embeddings", base.get('embeddings'))
list_dir("SAM2", base.get('sam2'))

custom_nodes_dir = base.get('custom_nodes')
if custom_nodes_dir and os.path.exists(custom_nodes_dir):
    nodes = [d for d in os.listdir(custom_nodes_dir) if os.path.isdir(os.path.join(custom_nodes_dir, d))]
    print(f"\n📦 Custom Nodes: {len(nodes)} installed")
    for node in nodes:
        print(f"   - {node}")
else:
    print("\n📦 Custom Nodes: Not installed")

print("\n✅ Verification complete!")
